In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeRegressor
from sklearn import metrics
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV
from scipy.stats import randint as sp_randint

In [22]:
tempIemisions = pd.read_csv("DataSetFinal.csv", index_col=0)
dummiesCountry=pd.get_dummies(tempIemisions['Country'],drop_first=1)
dummiesGas=pd.get_dummies(tempIemisions['Type'],drop_first=1)
tempIemisions=pd.concat([tempIemisions,dummiesCountry,dummiesGas],axis=1)
tempIemisions=tempIemisions.drop(['Country', 'Type'], axis=1)


In [23]:
tempIemisions.describe()

,Data,Value,AverageTemperature,Mes,Austria,Belgium,Bulgaria,Croatia,Czech Republic,Denmark,...,Portugal,Romania,Slovakia,Slovenia,Spain,Sweden,United Kingdom,CH4,CO2,N2O
count,22356.000000,2.235600e+04,22356.000000,22356.000000,22356.000000,22356.000000,22356.000000,22356.000000,22356.000000,22356.000000,...,22356.000000,22356.000000,22356.000000,22356.000000,22356.000000,22356.000000,22356.000000,22356.000000,22356.000000,22356.000000
mean,2001.000000,6.270248e+04,9.230230,6.832797,0.037037,0.037037,0.037037,0.037037,0.037037,0.037037,...,0.037037,0.037037,0.037037,0.037037,0.037037,0.037037,0.037037,0.333333,0.333333,0.333333
std,6.633398,1.379788e+05,9.537685,3.484143,0.188857,0.188857,0.188857,0.188857,0.188857,0.188857,...,0.188857,0.188857,0.188857,0.188857,0.188857,0.188857,0.188857,0.471415,0.471415,0.471415
min,1990.000000,6.148900e+01,-34.665000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,1995.000000,5.287999e+03,3.819500,4.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,2001.000000,1.352975e+04,10.129500,7.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,2007.000000,5.538075e+04,16.188250,10.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,1.000000,1.000000
max,2012.000000,1.052238e+06,28.916000,13.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [29]:
#fem X totes les columnes menys temp i y nomes temp el axis=1 es per indicar
#que es una label de les columnes
X = tempIemisions.drop('AverageTemperature', axis=1)
y = tempIemisions['AverageTemperature']

#dividim el dataSet en test 20% i entrenament 80%
#el random_state es la llavor que s'utilitzarà per randomitzar 
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=2)
#creem l'algorisme del arbre de decisions i l'entrenem
regressor = DecisionTreeRegressor()
regressor.fit(X_train,y_train)

#fem les prediccions
y_pred = regressor.predict(X_test)
df=pd.DataFrame({'Actual':y_test, 'Predicted':y_pred})  
print('Mean Absolute Error:', metrics.mean_absolute_error(y_test, y_pred))  
print('Mean Squared Error:', metrics.mean_squared_error(y_test, y_pred))  
print('Root Mean Squared Error:', np.sqrt(metrics.mean_squared_error(y_test, y_pred)))
comparacio=pd.DataFrame({'Actual':y_test, 'Predicted':y_pred})
print (comparacio)

Mean Absolute Error: 0.6913741055456172
Mean Squared Error: 2.2452814930679788
Root Mean Squared Error: 1.498426338886226
       Actual  Predicted
21522  -0.050      2.230
17676  -0.346     -0.001
4346   -8.982    -10.625
3551   17.223     17.223
16963  13.206     13.206
21236  -2.852     -2.852
20678  17.767     17.767
1349   16.424     16.424
20471  22.966     22.966
20723  14.118     14.118
16544  20.113     20.113
12789  10.732     10.732
805    12.795     12.795
20290  23.970     23.185
2651   23.710     24.109
18089  19.203     19.203
689    -1.982     -1.982
130    16.398     16.398
9969    5.089      4.449
14582  12.675     12.675
16222  19.408     19.408
1574   14.090     14.090
9570    7.541      7.541
16686  22.776     22.776
380    14.867     14.867
3368   19.201     19.201
11291  23.004     23.004
19006   1.088      1.088
1712   21.193     21.955
13254   8.052      8.471
...       ...        ...
2498   17.140     17.140
12348  -3.868      1.160
9844   19.661     19.661
211

In [13]:
parameters = {'max_depth':range(10, 30, 3), 
              'min_samples_split':range(2,70,3),
              'min_samples_leaf':range(2,70,3)}
scoring = 'neg_mean_absolute_error'
regressor1 = DecisionTreeRegressor()
gscv=GridSearchCV(regressor1,parameters,scoring)
gscv.fit(X_train,y_train)
print('Millor combinacio:', gscv.best_params_)
y_pred = gscv.predict(X_test)
print('Mean Absolute Error:', metrics.mean_absolute_error(y_test, y_pred))  
print('Mean Squared Error:', metrics.mean_squared_error(y_test, y_pred))  
print('Root Mean Squared Error:', np.sqrt(metrics.mean_squared_error(y_test, y_pred))) 

Millor combinacio: {'max_depth': 28, 'min_samples_leaf': 2, 'min_samples_split': 5}
Mean Absolute Error: 0.8586586724806201
Mean Squared Error: 2.165184564747006
Root Mean Squared Error: 1.4714566132737337


In [22]:
parameters = {'max_depth':[5,10,15,20,25,30,35], 
              'min_samples_split':sp_randint(2, 150),
              'min_samples_leaf':sp_randint(2, 150)}
scoring = 'neg_mean_absolute_error'
n_iter=100
regressor2 = DecisionTreeRegressor()
rscv=RandomizedSearchCV(regressor2,parameters,n_iter,scoring)
rscv.fit(X_train,y_train)
print('Millor combinacio:', rscv.best_params_)
y_pred = rscv.predict(X_test)
print('Mean Absolute Error:', metrics.mean_absolute_error(y_test, y_pred))  
print('Mean Squared Error:', metrics.mean_squared_error(y_test, y_pred))  
print('Root Mean Squared Error:', np.sqrt(metrics.mean_squared_error(y_test, y_pred))) 

Millor combinacio: {'max_depth': 20, 'min_samples_leaf': 10, 'min_samples_split': 28}
Mean Absolute Error: 1.1961516440095525
Mean Squared Error: 2.5948032398092025
Root Mean Squared Error: 1.6108392967050444
